In [68]:
import glob
import pandas as pd
import bs4 as bs
from bs4 import BeautifulSoup
import lxml as lx
from bs4 import NavigableString
#add inscriptions source
#known bugs:
#includes whitespace chars in df. also includes english descriptions sometimes,
#problem may be in misuse of epidoc format? see eg iAph080056

#todo:
#functionize get-date-info section


In [126]:
xml_filenames=glob.glob("*.xml")
df=pd.DataFrame(xml_filenames,columns=['xml_filenames'])
#df['xml_filenames']=xml_filenames

strings=[]
notBefore=[]
notAfter=[]
exact=[]
cert=[]

for filenm in xml_filenames:
    infile = open(filenm,"r")
    contents = infile.read()
    soup = BeautifulSoup(contents, "xml")

    
    #get strings
    
    edition=soup.find(type='edition')
    
    if edition is None:
        strings.append("W")
    elif 'lang' not in edition.attrs: #no relevant text
        strings.append("W")
        print(filenm, ' found no lang')
    elif edition['lang']=='la': #is latin
        strings.append("W")
        print(filenm, ' found the latin')
    else:
        this_string=""
        text=edition.find_all(['ab','lg'])
        for section in text:
            for child in section.descendants:
                if child.parent.name == "note" or child.name=="note":
                    continue
                elif type(child) is NavigableString and child.string is not None:
                    #if child.parent.name != "ab":
                    if child.string != "No text" or "o text":
                        this_string=this_string+child.string
        if this_string is not "":
            strings.append(this_string)
        else:
            strings.append("W")
            
            
    #get date info
    
    tag=soup.find(notBefore=True)
    if tag is None:
        notBefore.append('no value')
    else:
        notBefore.append(tag.get('notBefore'))
    tag=soup.find(notAfter=True)
    if tag is None:
        notAfter.append('no value')
    else:
        notAfter.append(tag.get('notAfter'))
    tag=soup.find(exact=True)
    if tag is None:
        exact.append('no value')
    else:
        exact.append(tag.get('exact'))
    tag=soup.find(cert=True)
    if tag is None:                #note none is an acceptable value in this category
        cert.append('no value')
    else:             
        cert.append(tag.get('cert'))


    
        
        
#runtime about 20 seconds on home laptop with 1505 files

iAph020524.xml  found the latin
iAph010201.xml  found the latin
iAph040304.xml  found the latin
iAph110063.xml  found the latin
iAph140005.xml  found no lang


In [131]:
print('expect ', len(strings), ' columns')
df['string']=strings
df['date_notBefore']=notBefore
df['date_notAfter']=notAfter
df['date_exact']=exact
df['date_cert']=cert



df.to_csv('collected_inscription_data.csv')
df

expect  1505  columns


,xml_filenames,string,date_notBefore,date_notAfter,date_exact,date_cert
0,iAph150305.xml,ακα εδεμο ωζαινο κεῖμαι δυ ου σεμνου νοσοτ,0101,0300,none,low
1,iAph120632.xml,ἡ σορὸς καὶ αἱ ὑποκείμεναι ἰσῶσται τέσσαρες κ...,0101,0300,none,no value
2,iAph090111.xml,σωτῆρι καὶ εὐεργέτῃ,0001,0300,none,no value
3,iAph080072.xml,\n\nχιου,0101,0600,none,no value
4,iAph080216.xml,ΛΗΣ,0001,0100,none,low
5,iAph110065.xml,ἕτερος δὲ οὐδεὶς ἕξει ἐξουσίαν ἐνθάψε τινὰ ἐ...,0301,0400,none,low
6,iAph090039.xml,Ἰουλίαν Σεβαστὴν Σεβαστοῦ θυγατέρα Ἥραν,0014,0069,both,no value
7,iAph120209.xml,ονισαπεθεμην,0101,0300,none,low
8,iAph050107.xml,ΩΝ,0001,0400,none,no value
9,iAph020509.xml,ΕΥΤ,0001,0300,none,no value


In [132]:
df[df['xml_filenames']=='iAph010003.xml']
#test for inclusion of strings from all ab sections in "type='edition'"
#(010003 has two)

,xml_filenames,string,date_notBefore,date_notAfter,date_exact,date_cert
463,iAph010003.xml,λαιιβ οἱ οἰκοδόμοι. ἐπιμεληθέντος,0001,1000,no value,low


In [129]:
#df.sort_values(by='date_notBefore',axis=0)
df[df['xml_filenames']=='iAph080270.xml']
#no ab section. extensive grk text in 'lg'.

,xml_filenames,string,date_notBefore,date_notAfter,date_exact,date_cert
88,iAph080270.xml,\n\n τύμβος Ἰορδά\nνοιο τὸν Εὐλαλί\nου ἀπὸ φύτ...,0401,0600,none,low


In [130]:
#df.sort_values(by='date_notBefore',axis=0)
df[df['xml_filenames']=='iAph100006.xml']
#both greek and english sections in 'edition'. english in 'note'.


,xml_filenames,string,date_notBefore,date_notAfter,date_exact,date_cert
374,iAph100006.xml,μοστρατου\n \nΒ\nΙ\nΟ\nΙ\nΛ σιέων \n\nἈπολλω...,0101,0500,none,no value
